# Домашнее задание №2

**Максимальная оценка за работу `min(X, 10)`, где X - суммарный балл за hw2 из 12 возможных.**

## Часть 1. ML workflow (**всего 5 баллов**)

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from itertools import product

### Загрузим данные для работы.

In [2]:
df = pd.read_csv("winequality-red.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Будем решать задачу регрессии: необходимо предсказать качество вина на основе его характеристик

### Шаг 1.  (**0.2 балла**)
Создайте матрицу X объект-признак и целевой вектор y ("quality")

In [3]:
X = df.drop('quality', axis= 1)
y = df['quality']
X.shape

(1599, 11)

### Шаг 2. (**0.2 балла**)
Разбейте данные на train и test (доля тестовых данных - 30%).

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=69)

### Шаг 3. (**0.2 балла**)
Обучите линейную регрессию на тренировочных данных и сделайте предсказания на train и на test.

In [5]:
model_regression = LinearRegression()
model_regression.fit(X_train, y_train)

y_pred_test = model_regression.predict(X_test)

y_pred_train = model_regression.predict(X_train)


### Шаг 4. (**0.4 балла**)
Выведите на экран ошибку MSE на train и на test, затем выведите на экран ошибку r2 на train и test.  

In [6]:
print('mse on test: {}'.format(mean_squared_error(y_test, y_pred_test)))

print('mse on train: {}'.format(mean_squared_error(y_train, y_pred_train)))

print('R2 on test: {}'.format(r2_score(y_test, y_pred_test)))

print('R2 on train: {}'.format(r2_score(y_train, y_pred_train)))

mse on test: 0.4624392804199679
mse on train: 0.398865506558818
R2 on test: 0.3693860519380159
R2 on train: 0.3525270082240314


### Шаг 5. (**0.5 балла**)
Вычислите среднее качество (r2) модели на кросс-валидации с k=5 фолдами.

In [7]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model_regression, X, y, cv=kf, scoring='r2')
mean_r2 = np.mean(scores)
print(f'Среднее значение R² на кросс-валидации: {mean_r2}')

Среднее значение R² на кросс-валидации: 0.34241461809490154


### Шаг 6.  (**0.5 балла**)
Теперь примените линейную регрессию с L1-регуляризацией (Lasso) для данной задачи. Объявите модель и подберите параметр регуляризации alpha по сетке. Ищите alpha в диапазоне (0.1, 1.1) с шагом 0.1.

Осуществите подбор параметра alpha по тренировочным данным (Xtrain, ytrain).

In [8]:
from sklearn.linear_model import Lasso

alpha_range = np.arange(0.1, 1.1, 0.1)

model_lasso = Lasso()

param_grid = {'alpha': alpha_range}
grid_search = GridSearchCV(model_lasso, param_grid, cv=5, scoring='r2')

grid_search.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])},
             scoring='r2')

### Шаг 7.  (**0.5 балла**)
Выведите наилучший алгоритм и наилучшее качество по результатам подбора alpha (best_estimator_ и best_score_).

In [9]:
best_alpha = grid_search.best_params_['alpha']
best_r2 = grid_search.best_score_
best_estimator = grid_search.best_estimator_

print(f'Лучший R2 на кросс-валидации: {best_r2}')
print(f'Лучший алгоритм: {best_estimator}')

Лучший R2 на кросс-валидации: 0.21048122732434527
Лучший алгоритм: Lasso(alpha=0.1)


### Шаг 8.  (**0.5 балла**)

С помощью найденного best_estimator_ сделайте предсказание на тестовых данных и выведите на экран r2-score на тесте.

In [10]:
y_pred_test = grid_search.best_estimator_.predict(X_test)


r2_test = r2_score(y_test, y_pred_test)

print(f'R2 на тестовых данных: {r2_test}')

R2 на тестовых данных: 0.23438906681633642


### Шаг 9.  (**0.5 балла**)

Попробуем улучшить качество модели за счет добавления полиномиальных признаков. Создайте pipeline, состоящий из добавления полиномиальных признаков степени 2, а затем применения линейной регрессии.

Затем вычислите r2-score этой модели на кросс валидации с пятью фолдами.

In [11]:
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2)),
    ('linear', LinearRegression())
])

kf = KFold(n_splits=5, shuffle=True, random_state=42)


scores = cross_val_score(pipeline, X, y, cv=kf, scoring='r2')


mean_r2 = np.mean(scores)


print(f'Среднее значение R2 на кросс-валидации: {mean_r2}')


Среднее значение R2 на кросс-валидации: 0.3300183838715737


### Шаг 10.  (**0.5 балла**)
Обучите модель (pipeline) на тренировочных данных и сделайте предсказания для train и test, затем выведите на экран r2-score и MSE на тренировочных и на тестовых данных.

In [12]:


pipeline.fit(X_train, y_train)


y_train_pred = pipeline.predict(X_train)


y_test_pred = pipeline.predict(X_test)


r2_train = r2_score(y_train, y_train_pred)
mse_train = mean_squared_error(y_train, y_train_pred)


r2_test = r2_score(y_test, y_test_pred)
mse_test = mean_squared_error(y_test, y_test_pred)


print(f'R2 на тренировочных данных: {r2_train}')
print(f'MSE на тренировочных данных: {mse_train}')
print(f'R2 на тестовых данных: {r2_test}')
print(f'MSE на тестовых данных: {mse_test}')

R2 на тренировочных данных: -1.869842580885987
MSE на тренировочных данных: 1.767921178657016
R2 на тестовых данных: -1.7407780900019136
MSE на тестовых данных: 2.009856349715119


### Сделайте выводы. Для этого ответьте на вопросы: (**1 балл**)

1) Хорошее ли качество показала исходная модель (линейная регрессия без регуляризации)? Является ли эта модель переобученной?

2) Помогла ли L1-регуляризация улучшить качество модели?

3) Помогло ли добавление полиномов второй степени улучшить качество модели? Как добавление новых признаков повлияло на переобучение?

1) Обычная линейная регрессия показала среднее качество, нельзя сказать, что она переобучилась.

2) L1-регуляризация снизила качество модели, так как R2 с 0.369 (линейная модель) упал до 0.234.Регуляризация обычно помогает в ситуациях, когда модель переобучена, но здесь исходная модель не страдала от переобучения, а Lasso добавила излишние штрафы, что снизило качество предсказаний.

3) Добавление полиномиальных признаков резко снизила качество модели, r2 вообще стал отрицательным. Полиномы второго порядка привели к слишком сложной модели, которая не смогла хорошо обобщить данные на тесте.



### *Попытайтесь улучшить модель (добейтесь наилучшего качества) - можно использовать любые методы.

При улучшении качества r2 на 0.1-0.2 +1 балл, при большем улучшении +2 балла (дополнительно к 5 баллам за основную часть).

In [13]:
from sklearn.ensemble import GradientBoostingRegressor

gb = GradientBoostingRegressor(random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grid_gb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.5, 0.1],
    'max_depth': [3, 5],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [3, 5]
}

# Поиск по сетке
grid_search_gb = GridSearchCV(gb, param_grid_gb, cv=10, scoring='r2', n_jobs=-1)
grid_search_gb.fit(X_train_scaled, y_train)

best_gb = grid_search_gb.best_estimator_

y_train_pred_gb = best_gb.predict(X_train_scaled)
y_test_pred_gb = best_gb.predict(X_test_scaled)

r2_train_gb = r2_score(y_train, y_train_pred_gb)
r2_test_gb = r2_score(y_test, y_test_pred_gb)
print(f'GradientBoosting - R2 на тренировке: {r2_train_gb}')
print(f'GradientBoosting - R2 на тесте: {r2_test_gb}')

GradientBoosting - R2 на тренировке: 0.8797877457002653
GradientBoosting - R2 на тесте: 0.47102394852148355


## Часть 2. Target encoding (**всего 5 баллов**)

В этом части домашнего задания вы будете работать с выборкой `1C`. Вам нужно посчитать счетчики для `item_id` четырьмя способами:

    1) При помощи KFold схемы;  
    2) При помощи Leave-one-out схемы;
    3) При помощи smoothing схемы;
    4) При помощи expanding mean схемы.

### Подготовка данных

In [14]:
sales = pd.read_csv('sales_train.csv.gz')
sales.columns = ['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'target']
sales

,date,date_block_num,shop_id,item_id,item_price,target
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [15]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = []
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'target':'sum'})

#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

### Mean encodings без регуляризации

После проделанной технической работы, мы готовы посчитать счетчики для переменной `item_id`.

Ниже приведены две реализации подсчета счетчиков без регуляризации. Можно использовать данный код в качестве стартовой точки для реализации различных техник регуляризации.

#### Способ 1

In [16]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True)

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621764


C:\Users\Булат\AppData\Local\Temp\ipykernel_13388\2599960635.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_data['item_target_enc'].fillna(0.3343, inplace=True)


#### Способ 2

In [17]:
'''
     Differently to `.target.mean()` function `transform`
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True)

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621764


C:\Users\Булат\AppData\Local\Temp\ipykernel_13388\2877932742.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_data['item_target_enc'].fillna(0.3343, inplace=True)


###  KFold схема (**1.25 балла**)

Необходимо реализовать Kfold схему с пятью фолдами. Используйте KFold(5) из sklearn.model_selection.

1. Разбейте данные на 5 фолдов при помощи `sklearn.model_selection.KFold` с параметром `shuffle=False`.
2. Проитерируйтесь по фолдам: используйте 4 обучающих фолда для подсчета средних значений таргета по `item_id` и заполните этими значениями валидационный фолд на каждой итерации.

Обратите внимание на **Способ 1** из примера. В частности, изучите, как работают функции map и pd.Series.map. Они довольно полезны во многих ситуациях.

In [18]:
kf = KFold(n_splits=5, shuffle=False)
all_data['item_target_enc_kfold'] = 0  

for train_idx, valid_idx in kf.split(all_data):
    
    train_data, valid_data = all_data.iloc[train_idx], all_data.iloc[valid_idx]
    
    
    item_id_target_mean = train_data.groupby('item_id')['target'].mean()
    
    
    all_data.loc[valid_idx, 'item_target_enc_kfold'] = valid_data['item_id'].map(item_id_target_mean)


all_data['item_target_enc_kfold'].fillna(0.3343, inplace=True)


encoded_feature_kfold = all_data['item_target_enc_kfold'].values
print(np.corrcoef(all_data['target'].values, encoded_feature_kfold)[0][1])

C:\Users\Булат\AppData\Local\Temp\ipykernel_13388\976808614.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.08474576 0.02158273        nan ...        nan        nan        nan]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  all_data.loc[valid_idx, 'item_target_enc_kfold'] = valid_data['item_id'].map(item_id_target_mean)


0.40923559668440646


C:\Users\Булат\AppData\Local\Temp\ipykernel_13388\976808614.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_data['item_target_enc_kfold'].fillna(0.3343, inplace=True)


Ожидаемый ответ 0.4165

### Leave-one-out схема (**1.25 балла**)

Необходимо реализовать leave-one-out схему . Учтите, если вы запустите код из первого задания, задав количество фолдов такое же как размер выборки, то вы, вероятно, получите правильный ответ, но ждать будете очень-очень долго.

Для более быстрой реализации подсчета среднего таргета на всех объектах, кроме одного, вы можете:

1. Вычислить суммарный таргет по всем объектам.
2. Вычесть таргет конкретного объекта и разделить результат на `n_objects - 1`.

Заметим, что пункт `1.` следует сделать для всех объектов. Также заметим, что пункт `2.` может быть реализован без циклов `for`.

Здесь может оказаться полезной функция .transform из **Способа 2** из примера.

In [19]:
item_id_sum = all_data.groupby('item_id')['target'].transform('sum')
item_id_count = all_data.groupby('item_id')['target'].transform('count')


all_data['item_target_enc_loo'] = (item_id_sum - all_data['target']) / (item_id_count - 1)


all_data['item_target_enc_loo'].fillna(0.3343, inplace=True)


encoded_feature_loo = all_data['item_target_enc_loo'].values
print(np.corrcoef(all_data['target'].values, encoded_feature_loo)[0][1])

0.4803848311293092


C:\Users\Булат\AppData\Local\Temp\ipykernel_13388\685291821.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_data['item_target_enc_loo'].fillna(0.3343, inplace=True)


Ожидаемый ответ 0.4803

### Smoothing (**1.25 балла**)

Необходимо реализовать smoothing с $\alpha = 100$. Используйте формулу:

$\frac{mean(target) \cdot nrows + globalmean \cdot \alpha }{nrows + \alpha}$,

где $globalmean=0.3343$. Заметим, что `nrows` - это количество объектов, принадлежащих конкретной категории, а не количество строк в датасете.

In [20]:
globalmean = 0.3343
alpha = 100


item_id_mean = all_data.groupby('item_id')['target'].transform('mean')
item_id_count = all_data.groupby('item_id')['target'].transform('count')


all_data['item_target_enc_smooth'] = (item_id_mean * item_id_count + globalmean * alpha) / (item_id_count + alpha)


encoded_feature_smooth = all_data['item_target_enc_smooth'].values
print(np.corrcoef(all_data['target'].values, encoded_feature_smooth)[0][1])

0.48181987970972573


Ожидаемый ответ 0.4818

### Expanding mean схема (**1.25 балла**)

Необходимо реализовать *expanding mean* схему. Ее суть заключается в том, чтобы пройти по отсортированному в определенном порядке датасету (датасет сортируется в самом начале задания) и для подсчета счетчика для строки $m$ использовать строки от $0$ до $m-1$. Вам будет необходимо воспользоваться pandas функциями [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) и [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html).

In [21]:
all_data['cumsum'] = all_data.groupby('item_id')['target'].cumsum() - all_data['target']
all_data['cumcnt'] = all_data.groupby('item_id').cumcount()


all_data['item_target_enc_expanding'] = np.where(all_data['cumcnt'] > 0, all_data['cumsum'] / all_data['cumcnt'], 0.3343)

encoded_feature_expanding = all_data['item_target_enc_expanding'].values
print(np.corrcoef(all_data['target'].values, encoded_feature_expanding)[0][1])

0.5025245211081716


Ожидаемый ответ 0.5025